In [60]:
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

### Manuelle Definition des Preprocessors und Mappings
Die folgenden Werte müssen aus dem Trainingsprozess generiert und hier eingefügt werden.

In [61]:
# === BEGINN DER WERTE, DIE AUS DEM TRAININGSSKRIPT GENERIERT WERDEN ===
# Diese Werte müssen Sie mit den tatsächlichen Ausgaben Ihres Trainingsskripts ersetzen!
label_to_score_map_notebook = {
    0: (3, 0),
    1: (3, 1),
    2: (3, 2),
    3: (0, 3),
    4: (1, 3),
    5: (2, 3),
}
N_CLASSES_NOTEBOOK = len(label_to_score_map_notebook)

numerical_features_notebook = ['P1_Rank', 'P1_Pts', 'P2_Rank', 'P2_Pts', 'Best_of']
categorical_features_notebook = ['Surface']

# MinMaxScaler Parameter (BEISPIELWERTE - ERSETZEN!)
min_scaler_min_ = np.array([  1.   ,  45.   ,   1.   ,  45.   ,   5.   ])
min_scaler_scale_ = np.array([9.54198473e-04, 9.05469033e-05, 5.41418517e-04, 9.04649900e-05,1.00000000e+00])

# OneHotEncoder Parameter (BEISPIELWERTE - ERSETZEN!)
onehot_categories_ = [
    np.array(['Clay', 'Grass', 'Hard'], dtype=object) # Reihenfolge und Inhalt aus Training!
]
input_shape_model_notebook = (8,) # BEISPIEL: 5 numerisch + 3 one-hot (Clay, Hard, Grass). Anpassen!
# === ENDE DER WERTE, DIE AUS DEM TRAININGSSKRIPT GENERIERT WERDEN ===

# Preprocessor manuell erstellen und konfigurieren
manual_scaler = MinMaxScaler()
manual_scaler.min_ = min_scaler_min_
manual_scaler.scale_ = min_scaler_scale_
manual_scaler.n_features_in_ = len(numerical_features_notebook)

manual_onehot = OneHotEncoder(categories=onehot_categories_, handle_unknown='ignore', sparse_output=False)
dummy_categorical_data = {col: [onehot_categories_[i][0]] for i, col in enumerate(categorical_features_notebook)}
dummy_df_for_onehot_fit = pd.DataFrame(dummy_categorical_data, columns=categorical_features_notebook)
manual_onehot.fit(dummy_df_for_onehot_fit)

preprocessor_notebook = ColumnTransformer([
    ('numerical', manual_scaler, numerical_features_notebook),
    ('categorical', manual_onehot, categorical_features_notebook)
], remainder='passthrough')

dummy_data_for_ct_fit = {col: [0] for col in numerical_features_notebook}
for i, col in enumerate(categorical_features_notebook):
    dummy_data_for_ct_fit[col] = [onehot_categories_[i][0]]
column_order_for_df_ct_fit = numerical_features_notebook + categorical_features_notebook
dummy_df_for_ct_fit = pd.DataFrame(dummy_data_for_ct_fit, columns=column_order_for_df_ct_fit)
preprocessor_notebook.fit(dummy_df_for_ct_fit)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['P1_Rank', 'P1_Pts', 'P2_Rank', 'P2_Pts',
                                  'Best_of']),
                                ('categorical',
                                 OneHotEncoder(categories=[array(['Clay', 'Grass', 'Hard'], dtype=object)],
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 ['Surface'])])

In [62]:
model_path = "LEM_tennis_set_predictor_final.keras" 
loaded_model = load_model(model_path)

In [63]:
def predict_sets(X_processed_for_model, p1_rank_raw, p2_rank_raw):
    """Erwartet bereits prozessierte Daten für das Modell und die rohen Ränge für Fallback."""
    y_pred_proba = loaded_model.predict(X_processed_for_model)
    predicted_label = np.argmax(y_pred_proba, axis=1)[0]
    predicted_score = label_to_score_map_notebook[predicted_label]

    # Konsistenzprüfung für Best-of-5 (da X immer Best of 5 sein wird für die Challenge)
    s1, s2 = predicted_score
    # Gewinner muss 3 Sätze haben, Verlierer weniger als 3. Gesamtzahl Sätze zwischen 3 und 5.
    is_valid_bo5 = ((s1 == 3 and 0 <= s2 < 3) or (s2 == 3 and 0 <= s1 < 3)) and (3 <= s1 + s2 <= 5)

    if is_valid_bo5:
        return list(predicted_score)
    else:
        # Fallback-Logik: Wenn die Vorhersage ungültig ist,
        # gib eine plausible Standardvorhersage zurück (z.B. Spieler mit besserem Rang gewinnt 3:0 oder 3:1).
        # Dies ist eine sehr einfache Heuristik.
        print(f"WARNUNG: Ungültige Vorhersage {predicted_score}. Fallback wird verwendet.")
        if p1_rank_raw < p2_rank_raw: # Spieler 1 hat besseres Ranking (niedriger ist besser)
            return [3, 0] 
        elif p2_rank_raw < p1_rank_raw: # Spieler 2 hat besseres Ranking
            return [0, 3]
        else: # Ränge gleich, oder einer ist NaN - sicherer Fallback
            return [3, 0] # Oder eine andere zufällige plausible Wahl


# NICHT DIE STRUKTUR VERÄNDERN
In der folgenden Zelle muss die Struktur exakt so erhalten bleiben, wie sie im Kommentar aufgelistet ist. Es dürfen lediglich die Werte ersetzt werden, aber es müssen immer alle Werte in exakt dieser Reihenfolge angegeben werden. Sollten Sie die Werte noch bearbeiten müssen, tun Sie dies bitte in der Zelle darunter. Es dürfen keine weiteren Werte wie Betting Rates in die Evaluation einfließen.

In [64]:
# ATP, Location, Tournament, Date, Series, Court, Surface, Round, Best of, Winner, Loser, WRank, LRank, WPts, LPts
X = [1, "Paris", "Roland Garros", "26.05.2024", "Grand Slam", "Outdoor", "Clay", "Quarterfinals", 5, "Alcaraz C.", "Djokovic N.", 7, 1, 6579, 11200]

In [65]:
X = [1, "Paris", "Roland Garros", "26.05.2024", "Grand Slam", "Outdoor", "Clay", "Semifinals", 5, "Alcaraz C.", "Djokovic N.", 1, 3, 6815, 5955]

In [66]:
def prepare_data(X_single_match_raw_list):
    """
    Bereitet die Rohdaten für EIN Match für die Modellvorhersage vor.
    """
    try:
        p1_rank_raw = X_single_match_raw_list[11]
        p2_rank_raw = X_single_match_raw_list[12]

        p1_rank = float(p1_rank_raw) if p1_rank_raw is not None else np.nan
        p2_rank = float(p2_rank_raw) if p2_rank_raw is not None else np.nan 
        p1_pts = float(X_single_match_raw_list[13]) if X_single_match_raw_list[13] is not None else np.nan
        p2_pts = float(X_single_match_raw_list[14]) if X_single_match_raw_list[14] is not None else np.nan
        surface = X_single_match_raw_list[6]
        best_of = float(X_single_match_raw_list[8])
        
        # Fallback für NaN Ranks/Points (z.B. mit einem Durchschnitt oder sehr schlechtem Wert)
        # Hier verwenden wir einen hohen Rang und 0 Punkte als einfachen Fallback.
        # Die Skalierung sollte damit umgehen können, solange die Trainingsdaten auch NaNs (oder imputierte Werte) hatten.
        # Es ist wichtig, dass der Scaler mit der Möglichkeit von NaNs (oder den Imputationswerten) trainiert wurde.
        # Wenn der Scaler im Training nie NaNs gesehen hat, ist dies ein Problem.
        # Eine einfache Imputation hier:
        p1_rank = p1_rank if pd.notna(p1_rank) else 2000.0 # Hoher Rang als Fallback
        p2_rank = p2_rank if pd.notna(p2_rank) else 2000.0
        p1_pts = p1_pts if pd.notna(p1_pts) else 0.0
        p2_pts = p2_pts if pd.notna(p2_pts) else 0.0

        data_for_df = {
            'P1_Rank': [p1_rank],
            'P1_Pts': [p1_pts],
            'P2_Rank': [p2_rank],
            'P2_Pts': [p2_pts],
            'Best_of': [best_of],
            'Surface': [surface]
        }
        
        column_order_for_df = numerical_features_notebook + categorical_features_notebook
        X_df_eval = pd.DataFrame(data_for_df, columns=column_order_for_df)
        
        X_processed_eval = preprocessor_notebook.transform(X_df_eval)
        
        # Rückgabe der rohen Ränge für die Fallback-Logik in predict_sets
        return X_processed_eval, p1_rank_raw, p2_rank_raw 

    except Exception as e:
        print(f"Fehler in prepare_data für {X_single_match_raw_list}: {e}")
        # Die Variable input_shape_model_notebook sollte aus der Definitionszelle kommen
        num_output_features = input_shape_model_notebook[0]
        print(f"Fehlerbehandlung: Erzeuge Nullvektor der Länge {num_output_features}")
        # Gebe auch Fallback-Ränge zurück, die zu einer Standardvorhersage führen
        return np.zeros((1, num_output_features)), 1000.0, 1000.0 


In [67]:
# Hier wird ausgewertet.
prepared_X_data, p1_rank_raw, p2_rank_raw = prepare_data(X) 
print(f"Form von prepared_X_data: {prepared_X_data.shape}") 
prediction_output = predict_sets(prepared_X_data, p1_rank_raw, p2_rank_raw)
print(prediction_output)

Form von prepared_X_data: (1, 8)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
[3, 0]
